<a href="https://www.kaggle.com/code/samithsachidanandan/basic-regression-to-predict-fuel-efficiency-tf?scriptVersionId=277023572" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

np.set_printoptions(precision=3, suppress=True)

In [ ]:
import tensorflow as tf 

from tensorflow import keras 
from tensorflow.keras import layers 
from tensorflow.keras import preprocessing
from tensorflow.keras.layers import Normalization


In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                 'Acceleration', 'Model Year', 'Origin']

dataset =pd.read_csv(url, names=column_names, na_values='?',
                    comment='\t', sep=' ', skipinitialspace=True)

In [ ]:
dataset.tail()

In [ ]:
# clean data
dataset = dataset.dropna()

#convert categorical 'Origin' data into one-hot data 
origin = dataset.pop('Origin')
dataset['USA'] = (origin==1) *1
dataset['Europe'] = (origin ==2) *1
dataset['Japan'] = (origin ==3 ) * 1

In [ ]:
dataset.tail()

In [ ]:
# Split the data into train and test 

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

print(dataset.shape, train_dataset.shape, test_dataset.shape)
train_dataset.describe().transpose()

In [ ]:
# Split features from labels
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels =train_features.pop('MPG')
test_labels = test_features.pop('MPG') 



In [ ]:
def plot(feature, x=None, y=None):
    plt.figure(figsize=(10,8))
    plt.scatter(train_features[feature], train_labels, label='Data')
    if x is not None and y is not None:
        plt.plot(x,y, color='k', label='Predictions')

    plt.xlabel(feature)
    plt.ylabel('MPG')
    plt.legend()
    plt.show()

In [ ]:
plot('Horsepower')

In [ ]:
plot('Weight')

### Normalize 

In [ ]:
# Normalize
print(train_dataset.describe().transpose()[['mean', 'std']])

# Normalization
normalizer = Normalization()

# adapt to the data
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())



In [ ]:
first = np.array(train_features[:1])
print('First example:', first)
print('Normalized:', normalizer(first).numpy())

In [ ]:
feature = 'Horsepower'
single_feature = np.array(train_features[feature])
print(single_feature.shape, train_features.shape)

single_feature = single_feature.reshape(-1, 1)

# Normalization
single_feature_normalizer = Normalization()

# adapt to the data
single_feature_normalizer.adapt(single_feature)



### Model 

In [ ]:
single_feature_model = keras.models.Sequential([
    single_feature_normalizer,
    layers.Dense(units=1) 
])

In [ ]:
single_feature_model.summary()


### Loss and Optimizer 

In [ ]:
loss = keras.losses.MeanAbsoluteError()
optim = keras.optimizers.Adam(learning_rate=0.1)

single_feature_model.compile(optimizer=optim, loss=loss)

In [ ]:
history = single_feature_model.fit(
    train_features[[feature]],  
    train_labels,
    epochs=100,
    verbose=1,
    validation_split=0.2
)

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0,25])
    plt.xlabel('Epoch')
    plt.ylabel('Error[MPG]')
    plt.legend()
    plt.grid(True)

plot_loss(history)

In [ ]:
single_feature_model.evaluate(
    test_features[feature],
    test_labels, verbose=1)

In [ ]:
range_min = np.min(test_features[[feature]]) - 10
range_max = np.max(test_features[[feature]]) + 10
x = tf.linspace(range_min, range_max, 200)[:, tf.newaxis]
y = single_feature_model.predict(x)

plot(feature, x,y)

In [ ]:
dnn_model = keras.Sequential([
    single_feature_normalizer, 
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

In [ ]:
dnn_model.compile(loss=loss, 
                  optimizer=tf.keras.optimizers.Adam(0.001))

dnn_model.summary()

In [ ]:
dnn_model.fit(
    train_features[feature], train_labels, 

    validation_split=0.2,
    verbose=1,
    epochs=100,

    
)

In [ ]:
dnn_model.evaluate(test_features[feature], test_labels, verbose=1)

In [ ]:
# predict and plot
x = tf.linspace(range_min, range_max, 200)
y = dnn_model.predict(x)

plot(feature, x,y)

In [ ]:
# multiple inputs
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss=loss)

In [ ]:
linear_model.fit(
    train_features, train_labels, 
    epochs=100,
    verbose=1,
    
    validation_split = 0.2)

In [ ]:

linear_model.evaluate(
    test_features, test_labels, verbose=1)